In [72]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

In [73]:
## diccionaries
catalogo = pd.read_excel("../BaseDatos.xlsx", sheet_name="Catalogo")
catalogo = catalogo[["Articulo", "Precio", "Costo", "TiempoVidaAnaquel", "Tamaño_Surtido"]].dropna()
for i in catalogo.columns:
    catalogo.rename(columns={i: i.lower().replace("ñ", "n")}, inplace=True)

catalogo.rename(columns={"articulo":"sku"}, inplace=True)
catalogo["ganancia_uni"] = catalogo["precio"]-catalogo["costo"]
catalogo["pro_ganacia"] = catalogo["ganancia_uni"]/catalogo["costo"]
catalogo = catalogo[["sku", "tamano_surtido", "ganancia_uni", "costo"]]
catalogo.rename(columns={"sku":"SKU"}, inplace=True)

catalogo.head()

,SKU,tamano_surtido,ganancia_uni,costo
0,Art01,1.0,2.33,5.44
1,Art02,3.0,10.27,15.40
2,Art03,2.0,9.53,17.70
3,Art04,1.0,10.24,10.25
4,Art05,1.0,7.00,16.35


In [78]:
vent_col = ['SKU', 'LOC', 'w_1', 'w_2', 'w_3', 'w_4', 'w_5',
       'w_6', 'w_7', 'w_8', 'w_9', 'w_10', 'w_11', 'w_12' ]

ventas = pd.read_csv("../data/ventas_sku_loc_week.csv")

ventas["week"] = ventas.groupby(["SKU","LOC"]).cumcount()+1
ventas["week"] = ventas["week"].apply(lambda x: "w_"+str(x))

ventas_pivot = ventas.pivot_table(columns="week",values="UNI", index=["SKU", "LOC"]).reset_index()

ventas_pivot= ventas_pivot[vent_col]

In [79]:
predict_ventas = ventas_pivot[["SKU", "LOC", "w_9", "w_10", "w_11", "w_12"]]

In [80]:
predict_ventas

week,SKU,LOC,w_9,w_10,w_11,w_12
0,Art01,LOC_001,9.0,13.0,8.0,6.0
1,Art01,LOC_002,3.0,8.0,3.0,2.0
2,Art01,LOC_003,4.0,6.0,9.0,12.0
3,Art01,LOC_004,9.0,18.0,8.0,12.0
4,Art01,LOC_005,9.0,6.0,13.0,5.0
...,...,...,...,...,...,...
3924,Art10,LOC_413,0.0,0.0,4.0,23.0
3925,Art10,LOC_414,0.0,0.0,2.0,1.0
3926,Art10,LOC_415,1.0,0.0,0.0,2.0
3927,Art10,LOC_416,1.0,3.0,1.0,2.0


In [96]:
def opti_ventas(predict_ventas, tamano_surtido, ganancia, perdida):
    if predict_ventas%tamano_surtido==0:
        return predict_ventas
    elif ((predict_ventas-np.floor(predict_ventas/tamano_surtido)*tamano_surtido)*ganancia)<((np.ceil(predict_ventas/tamano_surtido)*tamano_surtido-predict_ventas)*perdida):
        return np.ceil(predict_ventas/tamano_surtido)*tamano_surtido
    else:
        return np.floor(predict_ventas/tamano_surtido)*tamano_surtido

In [106]:
def optimizador_ventas(df, catalogo):
    df = pd.merge(df, catalogo, on="SKU", how="left")
    df["w_13_in"] = df[["w13", "tamano_surtido", "ganancia_uni", "costo"]].apply(
        lambda x: opti_ventas(x[0], x[1], x[2], x[3]), axis=1)
    df["w_13_in"] = df[["w_13_in", "tamano_surtido"]].apply(lambda x: x[1] if x[0]==0 else x[0], axis=1)

    df["w_14_in"] = df[["w14", "tamano_surtido", "ganancia_uni", "costo"]].apply(
        lambda x: opti_ventas(x[0], x[1], x[2], x[3]), axis=1)
    df["w_14_in"] = df[["w_14_in", "tamano_surtido"]].apply(lambda x: x[1] if x[0]==0 else x[0], axis=1)

    df["w_15_in"] = df[["w15", "tamano_surtido", "ganancia_uni", "costo"]].apply(
        lambda x: opti_ventas(x[0], x[1], x[2], x[3]), axis=1)
    df["w_15_in"] = df[["w_15_in", "tamano_surtido"]].apply(lambda x: x[1] if x[0]==0 else x[0], axis=1)


    df["w_16_in"] = df[["w16", "tamano_surtido", "ganancia_uni", "costo"]].apply(
        lambda x: opti_ventas(x[0], x[1], x[2], x[3]), axis=1)
    df["w_16_in"] = df[["w_16_in", "tamano_surtido"]].apply(lambda x: x[1] if x[0]==0 else x[0], axis=1)
    return df 

In [115]:
forecast = pd.read_csv("../data/processed/forecast.csv")
forecast = forecast[["w13", "w14", "w15", "w16"]]
forecast = pd.concat([predict_ventas[["SKU", "LOC"]], forecast], axis=1)

In [116]:
forecast_opti = optimizador_ventas(forecast, catalogo)

In [118]:
forecast_results = forecast_opti[["SKU", "LOC", "w_13_in", "w_14_in", "w_15_in", "w_16_in"]]

In [119]:
forecast_results.rename(columns={"w_13_in":"INV_IDEAL_24JUN19",
                             "w_14_in":"INV_IDEAL_01JUL19",
                             "w_15_in":"INV_IDEAL_08JUL19", 
                             "w_16_in": "INV_IDEAL_15JUL19"}, inplace=True)

/home/enrique/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [121]:
forecast_results.to_csv("../data/processed/forecasts_inv_24JUNI19-16JUL19.csv", index=False) 

In [122]:
forecast_results

,SKU,LOC,INV_IDEAL_24JUN19,INV_IDEAL_01JUL19,INV_IDEAL_08JUL19,INV_IDEAL_15JUL19
0,Art01,LOC_001,12.0,13.0,13.0,14.0
1,Art01,LOC_002,2.0,2.0,2.0,1.0
2,Art01,LOC_003,8.0,9.0,9.0,9.0
3,Art01,LOC_004,13.0,13.0,13.0,14.0
4,Art01,LOC_005,6.0,6.0,5.0,5.0
...,...,...,...,...,...,...
3924,Art10,LOC_413,2.0,2.0,2.0,2.0
3925,Art10,LOC_414,1.0,1.0,1.0,1.0
3926,Art10,LOC_415,1.0,1.0,1.0,1.0
3927,Art10,LOC_416,2.0,2.0,2.0,2.0
